In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import chromadb
import tiktoken
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)

load_dotenv()

## Basic RAG 

In [ ]:
file_path = "assets/bbva.pdf"
loader = PyPDFLoader(file_path)
pages = []

for page in loader.lazy_load():
    pages.append(page)

In [ ]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

system_prompt = """
You are a helpful assistant that can answer questions about the provided context.

Please cite the page number used to answer the question. Write the page number in the format "Page X" at the end of your answer. 

If the answer is not found in the context, please say so.
"""
user_message = """
Please answer the following question based on the context provided:

Question: {question}

Documents:
{documents}
"""

messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_message)]
context = ""
for i, page in enumerate(pages):
    context += f"--- PAGE {i + 1} ---\n{page.page_content}\n\n"


def get_response(context: dict):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message.format(**context)),
    ]
    response = model.invoke(messages)
    return response.content


question = "What is the main idea of the document?"
response = get_response({"question": question, "documents": context})
print(response)

In [ ]:
question = "What are the daily transaction limits?"
response = get_response({"question": question, "documents": context})
print(response)

In [ ]:
question = "How many cards can I have?"
response = get_response({"question": question, "documents": context})
print(response)

## RAG with vector search

### Create a vector database

In [ ]:
openai_ef = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))
vector_db = chromadb.PersistentClient()

try:
    collection = vector_db.delete_collection("bbva")
except:
    pass

collection = vector_db.create_collection("bbva", embedding_function=openai_ef)

### Split and index documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(pages)

In [ ]:
collection.add(
    documents=[split.page_content for split in all_splits],
    metadatas=[split.metadata for split in all_splits],
    ids=[str(i) for i in range(len(all_splits))],
)

### Query the database

In [ ]:
text = "What are the customer service channels?"
embeddings = openai_ef.embed_with_retries(text)
embeddings

In [ ]:
collection.query(query_texts=["What are the customer service channels?"], n_results=3)

In [ ]:
import numpy as np

text_1 = "What are the customer service channels?"
text_2 = "What are the customer service channels?"

embeddings_1 = openai_ef.embed_with_retries(text_1)[0]
embeddings_2 = openai_ef.embed_with_retries(text_2)[0]

cos_sim = np.dot(embeddings_1, embeddings_2) / (
    np.linalg.norm(embeddings_1) * np.linalg.norm(embeddings_2)
)
cos_sim

In [ ]:
embeddings_1

In [ ]:
embeddings_2

In [ ]:
collection.query(query_embeddings=embeddings, n_results=3)

In [ ]:
collection.query(
    query_texts=["What are the daily transaction limits?", "Is there a monthly limit?"],
    n_results=3,
)

### Generate a response

In [ ]:
from langsmith import traceable

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

system_prompt = """
You are a helpful assistant that can answer questions about the provided context.

Please cite the page number used to answer the question. Write the page number in the format "Page X" at the end of your answer. 

If the answer is not found in the context, please say so.
"""

user_message = """
Please answer the following question based on the context provided:

Question: {question}

Documents:
{documents}
"""


@traceable
def get_relevant_docs(question: str):
    relevant_docs = collection.query(query_texts=question, n_results=3)
    documents = relevant_docs["documents"][0]
    metadatas = relevant_docs["metadatas"][0]
    return [
        {"page_content": doc, "type": "Document", "metadata": metadata}
        for doc, metadata in zip(documents, metadatas)
    ]


def get_context(relevant_docs: list[dict]):
    context = ""
    for doc in relevant_docs:
        context += f"--- PAGE {doc['metadata']['page']} ---\n{doc['page_content']}\n\n"
    return context


@traceable
def get_messages(question: str, relevant_docs: dict):
    prompt_vars = {"question": question, "documents": get_context(relevant_docs)}
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message.format(**prompt_vars)),
    ]
    return messages


@traceable
def get_response(question: str):
    relevant_docs = get_relevant_docs(question)
    messages = get_messages(question, relevant_docs)
    response = model.invoke(messages)
    return response.content


question = "What are the customer service channels?"
response = get_response(question)
print(response)

# Excercise

Download this book and create a vector database with it: https://github.com/mlschmitt/classic-books-markdown/blob/main/Friedrich%20Nietzsche/Beyond%20Good%20and%20Evil.md

In [ ]:
loader = TextLoader("assets/book.md")
book_text = loader.load()[0].page_content

md_text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "header_1"), ("##", "header_2")]
)
all_splits = md_text_splitter.split_text(book_text)

In [ ]:
openai_ef = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"))
vector_db = chromadb.PersistentClient()

try:
    vector_db.delete_collection("book")
except Exception:
    pass

collection = vector_db.create_collection("book", embedding_function=openai_ef)

In [ ]:
try:
    collection.add(
        documents=[s.page_content for s in all_splits],
        ids=[str(i) for i in range(len(all_splits))],
        metadatas=[s.metadata for s in all_splits],
    )
except Exception as e:
    print(e)

In [ ]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
tokens = [len(encoding.encode(s.page_content)) for s in all_splits]
print(tokens)

In [ ]:
def get_first_n_tokens(text, n):
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
    tokens = encoding.encode(text)
    text_tokens = encoding.decode(tokens[:n])
    return text_tokens

In [ ]:
collection.add(
    documents=[get_first_n_tokens(s.page_content, 8191) for s in all_splits],
    ids=[str(i) for i in range(len(all_splits))],
    metadatas=[s.metadata for s in all_splits],
)

In [ ]:
def get_num_tokens(text):
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
    return len(encoding.encode(text))


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=8192,
    separators=["\n\n", "\n"],
    chunk_overlap=0,
    length_function=get_num_tokens,
)

adjusted_splits = text_splitter.split_documents(all_splits[1:])

In [ ]:
collection = vector_db.get_or_create_collection(name="book2")

collection.add(
    documents=[s.page_content for s in adjusted_splits],
    ids=[str(i) for i in range(len(adjusted_splits))],
    metadatas=[s.metadata for s in adjusted_splits],
)

In [ ]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

system_prompt = """
You are a helpful assistant that can answer questions about the provided context.

Please cite the chapter used to answer the question. Write the page number in the format "Chapter X" at the end of your answer. 

If the answer is not found in the context, please say so.
"""

user_message = """
Please answer the following question based on the context provided:

Question: {question}

Documents:
{documents}
"""


@traceable
def get_relevant_docs(question: str):
    docs = collection.query(
        query_texts=[question],
        n_results=3,
        include=["distances", "metadatas", "documents"],
    )
    documents = docs["documents"][0]
    metadatas = docs["metadatas"][0]
    return [
        {"page_content": doc, "type": "Document", "metadata": metadata}
        for doc, metadata in zip(documents, metadatas)
    ]


def get_context(relevant_docs):
    context = ""
    for doc in relevant_docs:
        context += f"--- {doc['metadata']['header_1']}, {doc['metadata']['header_2']} ---\n{doc['page_content']}\n\n"
    return context


def get_messages(question: str, relevant_docs: dict):
    prompt_vars = {"question": question, "documents": get_context(relevant_docs)}
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message.format(**prompt_vars)),
    ]
    return messages


@traceable
def get_response(question: str):
    relevant_docs = get_relevant_docs(question)
    messages = get_messages(question, relevant_docs)
    response = model.invoke(messages)
    return response.content


response = get_response("What is the will to Truth?")

In [ ]:
docs = get_relevant_docs("What is the will to Truth?")
for d in docs:
    print(d["metadata"])
    print(d["page_content"])
    print("-" * 100)